- soft dtw divergence : https://rtavenar.github.io/ml4ts_ensai/contents/align/softdtw.html
- contraindre sur la digonale
- comaparer L2 et DTW
- Autoencoder
- VAE
- Detection d'anomalies

In [1]:
import numpy as np
import torch
from dtw_soft import backward_recursion, soft_dtw, soft_dtw_batch, soft_min
from sdtw import SoftDTW
from sdtw.distance import SquaredEuclidean
from tslearn import metrics

# test soft_min

In [2]:
# create list of values
list_a = [9, 10, 0.5, 1, 2, 3, 4, 100, -1]
gamma = 1

# calculate softmin
_min = soft_min(list_a, gamma)
print(_min)

# calculate softmin with gamma = 0
_min = soft_min(list_a, 0)
print(_min)

tensor(-1.3600)
tensor(-1.)


## example with 1D signal

In [3]:
x_1d = torch.Tensor([1, 1, 56]).T
y_1d = torch.Tensor([1, 8, 1, 9, 1]).T

C:\Users\baptc\AppData\Local\Temp\ipykernel_5320\3965251754.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3618.)
  x_1d = torch.Tensor([1, 1, 56]).T


In [4]:
dist, R = soft_dtw(x_1d, y_1d, gamma=1.0)
dist.item()

Carefull : x and y are not D-dimensional > 1 features : added 2 dimensions


3135.51513671875

In [5]:
# convert to numpy
x_1d = x_1d.numpy().reshape(-1, 1)
y_1d = y_1d.numpy().reshape(-1, 1)

# method 1
D = SquaredEuclidean(x_1d, y_1d)
sdtw = SoftDTW(D, gamma=1.0)
value = sdtw.compute()
# gradient w.r.t. D, shape = [m, n], which is also the expected alignment matrix
print(value)

# method 2
metrics.soft_dtw(x_1d, y_1d, gamma=gamma)

3135.515093350212


3135.515093350212

# example with 2D data

In [6]:
x1 = torch.Tensor([[1, 1, 56], [2, 8, 0]]).T
y1 = torch.Tensor([[1, 8, 1, 9, 1], [5, 9, 14, 7, -1]]).T

In [7]:
dist, R = soft_dtw(x1, y1, gamma=1.0)
dist.item()

3186.0

In [8]:
x1 = np.array([[1, 1, 56], [2, 8, 0]]).T
y1 = np.array([[1, 8, 1, 9, 1], [5, 9, 14, 7, -1]]).T

In [9]:
D = SquaredEuclidean(x1, y1)
sdtw = SoftDTW(D, gamma=1.0)
value = sdtw.compute()
# gradient w.r.t. D, shape = [m, n], which is also the expected alignment matrix
print(value)

# method 2
metrics.soft_dtw(x1, y1, gamma=gamma)

3185.9998765978103


3185.9998765978103

# Batch

In [10]:
x1 = torch.Tensor([[1, 1, 56], [2, 8, 0]]).T
x2 = torch.Tensor([[1, 8, 1], [5, 9, 14]]).T
X = torch.stack([x1, x2])
y1 = torch.Tensor([[1, 8, 1, 9, 1], [5, 9, 14, 7, -1]]).T
y2 = torch.Tensor([[1, 5, 8, 1, 1], [5, 5, 14, 10, -1]]).T
Y = torch.stack([y1, y2])

In [11]:
soft_dtw_batch(X, Y, gamma=1.0)

tensor([3186.0000,  281.9999])

In [12]:
x1 = np.array([[1, 1, 56], [2, 8, 0]]).T
y1 = np.array([[1, 8, 1, 9, 1], [5, 9, 14, 7, -1]]).T

x2 = np.array([[1, 8, 1], [5, 9, 14]]).T
y2 = np.array([[1, 5, 8, 1, 1], [5, 5, 14, 10, -1]]).T

In [13]:
D = SquaredEuclidean(x1, y1)
sdtw = SoftDTW(D, gamma=1.0)
value = sdtw.compute()
# gradient w.r.t. D, shape = [m, n], which is also the expected alignment matrix
print(value)

# method 2
metrics.soft_dtw(x1, y1, gamma=gamma)

3185.9998765978103


3185.9998765978103

In [14]:
D = SquaredEuclidean(x2, y2)
sdtw = SoftDTW(D, gamma=1.0)
value = sdtw.compute()
# gradient w.r.t. D, shape = [m, n], which is also the expected alignment matrix
print(value)

# method 2
metrics.soft_dtw(x2, y2, gamma=gamma)

281.9998765977964


281.9998765977964

# Backward recursion

In [15]:
x = torch.Tensor([1, 1, 56]).T
y = torch.Tensor([1, 8, 1, 9, 1]).T

In [16]:
x.shape

torch.Size([3])

In [17]:
backward_recursion(x, y, gamma=1)

Carefull : x and y are not D-dimensional > 1 features : added 2 dimensions
Carefull : x and y are not D-dimensional > 1 features : added 2 dimensions


tensor([[1.0000, 0.6666, 0.5000, 0.1667, 0.0833],
        [0.1667, 0.3333, 0.6666, 0.8333, 0.5833],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0000]])

In [18]:
x = np.array([1, 1, 56]).reshape(-1, 1)
y = np.array([1, 8, 1, 9, 1]).reshape(-1, 1)

In [19]:
D = SquaredEuclidean(x, y)
sdtw = SoftDTW(D, gamma=1.0)
value = sdtw.compute()
# gradient w.r.t. D, shape = [m, n], which is also the expected alignment matrix
E = sdtw.grad()
E

array([[1.        , 0.66666667, 0.5       , 0.16666667, 0.08333333],
       [0.16666667, 0.33333333, 0.66666667, 0.83333333, 0.58333333],
       [0.        , 0.        , 0.        , 0.        , 1.        ]])